# Parameter Optimization

Run grid search or random search optimization to find optimal parameters.

## Configuration

Define the parameter grid and optimization settings below.


In [ ]:
# Configuration
strategy_name = 'spy_sma_cross'
start_date = '2020-01-01'
end_date = None
method = 'grid'  # 'grid' or 'random'
objective = 'sharpe'  # 'sharpe', 'sortino', 'total_return', 'calmar'
train_pct = 0.7  # Training data percentage

# Parameter grid (for grid search)
param_grid = {
    'strategy.fast_period': [5, 10, 15, 20],
    'strategy.slow_period': [30, 50, 100],
}

# For random search, use distributions:
# param_distributions = {
#     'strategy.fast_period': [5, 10, 15, 20],
#     'strategy.slow_period': [30, 50, 100],
# }
n_iter = 100  # For random search


In [ ]:
# Setup
import sys
from pathlib import Path
import pandas as pd
import json

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

from lib.optimize import grid_search, random_search


In [ ]:
# Run optimization
if method == 'grid':
    print(f"Running grid search for {strategy_name}...")
    results_df = grid_search(
        strategy_name=strategy_name,
        param_grid=param_grid,
        start_date=start_date,
        end_date=end_date,
        objective=objective,
        train_pct=train_pct
    )
else:
    print(f"Running random search for {strategy_name}...")
    results_df = random_search(
        strategy_name=strategy_name,
        param_distributions=param_grid,  # Use same grid as distributions
        n_iter=n_iter,
        start_date=start_date,
        end_date=end_date,
        objective=objective,
        train_pct=train_pct
    )

print(f"\n✓ Optimization complete: {len(results_df)} combinations tested")


In [ ]:
# Display results
test_obj_col = f'test_{objective}'
if test_obj_col in results_df.columns:
    best_idx = results_df[test_obj_col].idxmax()
    best_row = results_df.loc[best_idx]
    
    print("=" * 60)
    print("BEST PARAMETERS")
    print("=" * 60)
    for param_name in param_grid.keys():
        if param_name in best_row:
            print(f"{param_name}: {best_row[param_name]}")
    
    print(f"\nPerformance:")
    print(f"  Train {objective}: {best_row.get(f'train_{objective}', 0):.4f}")
    print(f"  Test {objective}: {best_row.get(test_obj_col, 0):.4f}")
    print(f"  Train Sharpe: {best_row.get('train_sharpe', 0):.4f}")
    print(f"  Test Sharpe: {best_row.get('test_sharpe', 0):.4f}")
    print("=" * 60)


In [ ]:
# Display top 10 results
print("\nTop 10 Results:")
print(results_df.nlargest(10, test_obj_col)[['train_' + objective, test_obj_col, 'train_sharpe', 'test_sharpe'] + list(param_grid.keys())])


In [ ]:
# Load overfit score
from lib.utils import get_project_root

results_base = get_project_root() / 'results' / strategy_name / 'latest'
overfit_file = results_base / 'overfit_score.json'

if overfit_file.exists():
    with open(overfit_file) as f:
        overfit = json.load(f)
    
    print("\n" + "=" * 60)
    print("OVERFIT ANALYSIS")
    print("=" * 60)
    print(f"Efficiency (OOS/IS): {overfit.get('efficiency', 0):.3f}")
    print(f"PBO: {overfit.get('pbo', 0):.2f}")
    print(f"Verdict: {overfit.get('verdict', 'unknown')}")
    print("=" * 60)


In [ ]:
# Display heatmap if 2 parameters
if len(param_grid) == 2:
    from IPython.display import Image, display
    
    heatmap_file = results_base / f'heatmap_{objective}.png'
    if heatmap_file.exists():
        display(Image(str(heatmap_file)))
    else:
        print("Heatmap not available (requires 2 parameters)")
